In [106]:
# Import librariesi
from bs4 import BeautifulSoup
import requests

import pandas as pd

### Step 1: Create a soup object from the home page

In [87]:
#define the url and initiate the request
ga_url = "https://pages.git.generalassemb.ly/rldaggie/for-scraping/"
req_main = requests.get(ga_url)

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [88]:
#status code of our request 
req_main.status_code

200

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [89]:
#our list of food
foods=[]

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [108]:
def get_menu(url, restaurant):
    
    '''pass the url (to parse) and the restaurant name needed for the list only'''
    
    my_req = requests.get(url, headers={'User-Agent': 'Custom'})
    
    if (my_req.status_code == 200):
        
        print("Processing..", url)
        my_soup = BeautifulSoup(my_req.text, 'lxml')

        table = my_soup.find('table', {"class":"table"})

        tbody = table.find('tbody')
        all_trs = tbody.find_all('tr')

        for tr in all_trs:
            td = tr.find_all('td')
            name = td[0].text
            category = td[1].text
            calories = td[2].text
            fats = td[3].text
            carbs = td[4].text
            rest_dic = {"restaurant": restaurant, "name": name,  "category": category, 
                                "calories" : calories,
                                "fats": fats,
                                "carbs": carbs
                               }
            foods.append(rest_dic)
    else:
        print("Error in getting request:", url)

In [109]:
main_soup = BeautifulSoup(req_main.text, 'lxml')
all_tds = main_soup.find_all('td')
for td in all_tds:
    a = td.find('a')    
    if a:
        nested_url = ga_url + a["href"]
        get_menu(nested_url, a.text)

Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/1.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/2.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/3.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/4.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/5.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/6.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/7.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/8.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/9.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/10.html
Processing.. https://pages.git.generalassemb.ly/rldaggie/for-scraping/restaurants/11.html
Processing.. https:

In [110]:
df = pd.DataFrame(foods)

In [111]:
df.head()

,restaurant,name,category,calories,fats,carbs
0,A&W Restaurants,Original Bacon Double Cheeseburger,Burgers,760,45,45
1,A&W Restaurants,Coney (Chili) Dog,Entrees,340,20,26
2,A&W Restaurants,Chili Fries,French Fries,370,15,49
3,A&W Restaurants,Strawberry Milkshake (small),Shakes,670,29,90
4,A&W Restaurants,A&WÂ® Root Beer Freeze (large),Shakes,820,18,150


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [112]:
df.to_csv("restaurant.menu.csv", index=False)